To install the DeepFace library, use the command: !pip install deepface.

In [1]:
!pip install deepface

The following code defines a function called face_verification, which uses the DeepFace library to verify whether two face images belong to the same person. Here’s a brief breakdown of the code:

- Imports: The function imports the DeepFace class from the DeepFace library, which provides tools for face recognition and verification.

- Function Definition: face_verification(image_path1, image_path2, model_name='VGG-Face', distance_metric='cosine') accepts two image paths as inputs along with optional parameters for the model name and distance metric. By default, it uses the 'VGG-Face' model and the 'cosine' distance metric.

- Verification Process: The function calls DeepFace.verify, passing the two image paths and specified parameters. This function performs the face verification and returns a result dictionary containing details such as the distance between embeddings and whether the two images are verified as the same person.

- Extracting Results: It extracts the distance and verified values from the result. The distance indicates how similar the two faces are based on the embeddings, while verified is a boolean indicating if they are considered the same person.

- Displaying Results: The function checks the verified value:
If True, it prints a success message with the distance and the verification threshold. If False, it prints a failure message with the distance and the threshold.

- Return Value: Finally, the function returns True if the verification is successful and False otherwise.

In [2]:
from deepface import DeepFace

def face_verification(image_path1, image_path2, model_name='VGG-Face', distance_metric='cosine'):
    # Use DeepFace's built-in verification and distance metric
    result = DeepFace.verify(image_path1, image_path2, model_name=model_name, distance_metric=distance_metric, enforce_detection=False)

    # Extract the relevant details
    dist = result['distance']
    verified = result['verified']

    # Display results
    if verified:
        print(f"Verification successful. Distance: {dist:.2f} (Threshold: {result['threshold']})")
        return True
    else:
        print(f"Verification not successful. Distance: {dist:.2f} (Threshold: {result['threshold']})")
        return False


Create a database of individuals:

In [3]:


database = {
     "trump": "/content/trump.jpg",
     "elon musk": "/content/elon_musk.jpg" ,
     "ghandhi" : "/content/ghandhi.jpg" ,
     "mohammad ali clay" : "/content/mohammad_ali.jpg" ,
     "winston churchill" : "/content/churchill.jpg" ,
     }



In [4]:
def face_recognition(image_path, database, model_name='VGG-Face', distance_metric='cosine'):
    """
    Performs face recognition to identify the person in the given image by comparing it with images in the database.

    Parameters:
    - image_path: str, path to the image to be recognized
    - database: dict, where keys are identities and values are the paths to their known images
    - model_name: str, the model to use for generating embeddings (default is 'VGG-Face')
    - distance_metric: str, the metric to calculate distance ('cosine', 'euclidean', etc.)
    - threshold: float, the threshold for verification (default is 0.68)

    Returns:
    - str: the identity of the recognized person, or "Unknown" if no match is found
    """
    recognized_identity = "Unknown"

    # Loop through the database to verify each identity
    for identity, known_image_path in database.items():
        verified = face_verification(image_path, known_image_path, model_name, distance_metric)
        if verified:
            recognized_identity = identity
            break


    return recognized_identity
        # Use the face_verification function to check each identity



In [5]:
image_path  = "/content/trump.jpg"
image_path_test = "/content/trump_test.jpg"
recognized_person = face_verification(image_path, image_path_test)
print( "person :" , recognized_person)

Verification successful. Distance: 0.58 (Threshold: 0.68)
person : True


In [6]:
image_path  = "/content/mohammad_ali.jpg"
image_path_test  = "/content/mohammad_ali_test.jpg"

recognized_person = face_verification(image_path, image_path_test)
print("person:" , recognized_person)

Verification successful. Distance: 0.65 (Threshold: 0.68)
person: True


In [7]:
image_path  = "/content/churchill.jpg"
image_path_test  = "/content/churchill_test.jpg"

recognized_person = face_verification(image_path, image_path_test)
print("person:" , recognized_person)

Verification successful. Distance: 0.21 (Threshold: 0.68)
person: True


In [8]:
image_path  = "/content/trump.jpg"
recognized_person = face_recognition(image_path, database)
print("person:" , recognized_person)

Verification successful. Distance: 0.00 (Threshold: 0.68)
person: trump


In [9]:
image_path  = "/content/ghandhi.jpg"
recognized_person = face_recognition(image_path, database)
print("person:" , recognized_person)

Verification not successful. Distance: 0.86 (Threshold: 0.68)
Verification not successful. Distance: 0.96 (Threshold: 0.68)
Verification successful. Distance: 0.00 (Threshold: 0.68)
person: ghandhi


In [10]:
image_path  = "/content/elon_musk.jpg"
recognized_person = face_recognition(image_path, database)
print("person:" , recognized_person)

Verification not successful. Distance: 0.90 (Threshold: 0.68)
Verification successful. Distance: -0.00 (Threshold: 0.68)
person: elon musk


In [13]:
import numpy as np
import time
models = ["VGG-Face", "Facenet", "ArcFace"]
distance_metric = 'cosine'
results = []

for model in models:
        print(f"Evaluating model: {model}")

        total_distance = []
        correct_matches = 0
        total_pairs = 0
        total_time = 0

        for identity1, image_path1 in database.items():
            for identity2, image_path2 in database.items():
                if identity1 != identity2:
                    total_pairs += 1

                    start_time = time.time()

                    result = DeepFace.verify(image_path1, image_path2, model_name=model, distance_metric=distance_metric, enforce_detection=False , threshold=0.4)

                    elapsed_time = time.time() - start_time
                    total_time += elapsed_time

                    dist = result['distance']
                    verified = result['verified']

                    total_distance.append(dist)

                    if not verified:
                        correct_matches += 1

                    print(f"{identity1} vs {identity2} - Distance: {dist:.2f}, Verified: {verified}, Time: {elapsed_time:.2f}s")

        # Average metrics
        avg_distance = np.mean(total_distance)
        accuracy = correct_matches / total_pairs
        avg_time = total_time / total_pairs

        results.append({
            "model": model,
            "avg_distance": avg_distance,
            "accuracy": accuracy,
            "avg_time": avg_time
        })



Evaluating model: VGG-Face
trump vs elon musk - Distance: 0.90, Verified: False, Time: 3.69s
trump vs ghandhi - Distance: 0.86, Verified: False, Time: 4.65s
trump vs mohammad ali clay - Distance: 0.91, Verified: False, Time: 4.02s
trump vs winston churchill - Distance: 0.81, Verified: False, Time: 2.80s
elon musk vs trump - Distance: 0.90, Verified: False, Time: 3.56s
elon musk vs ghandhi - Distance: 0.96, Verified: False, Time: 4.54s
elon musk vs mohammad ali clay - Distance: 0.90, Verified: False, Time: 3.65s
elon musk vs winston churchill - Distance: 0.85, Verified: False, Time: 2.27s
ghandhi vs trump - Distance: 0.86, Verified: False, Time: 4.60s
ghandhi vs elon musk - Distance: 0.96, Verified: False, Time: 5.28s
ghandhi vs mohammad ali clay - Distance: 0.96, Verified: False, Time: 3.85s
ghandhi vs winston churchill - Distance: 0.90, Verified: False, Time: 3.36s
mohammad ali clay vs trump - Distance: 0.91, Verified: False, Time: 2.67s
mohammad ali clay vs elon musk - Distance: 0.90

In [12]:
#brightness:
from PIL import Image, ImageEnhance
import time
def adjust_lighting(image_path, modified_path, brightness=0):
    image = Image.open(image_path)
    enhancer = ImageEnhance.Brightness(image)
    enhanced_image = enhancer.enhance(brightness)
    enhanced_image.save(modified_path)
    return modified_path

def rotate_image(image_path, modified_path, angle):
    image = Image.open(image_path)
    rotated_image = image.rotate(angle)
    rotated_image.save(modified_path)
    return modified_path

def resize_image(image_path, modified_path, size):
    image = Image.open(image_path)
    resized_image = image.resize(size)
    resized_image.save(modified_path)
    return modified_path

models = ['VGG-Face', 'Facenet', 'ArcFace']

# Test brightness
for model in models:
    print(f"Testing model: {model}")
    start_time = time.time()  # Start timer
    for brightness in [0.1, 0.3, 0.5, 0.8]:
        modified_image_path = adjust_lighting("/content/trump.jpg", modified_path="trump_modified.jpg", brightness=brightness)
        recognized_person = face_recognition(modified_image_path, database, model_name=model)
        print(f"Recognized person (brightness: {brightness}, model: {model}):", recognized_person)

# Test rotation
for model in models:
    print(f"\nTesting model: {model}")
    start_time = time.time()  # Start timer
    for angle in [10, 20, 180]:
        modified_image_path = rotate_image("/content/trump.jpg", modified_path="trump_modified.jpg", angle=angle)
        recognized_person = face_recognition(modified_image_path, database, model_name=model)
        print(f"Recognized person (angle: {angle}, model: {model}):", recognized_person)


# Test resizing
for model in models:
    print(f"\nTesting model: {model}")
    start_time = time.time()  # Start timer
    for size in [(900, 900), (1100, 1420), (1200, 1520)]:
        modified_image_path = resize_image("/content/trump.jpg", modified_path="trump_modified.jpg", size=size)
        recognized_person = face_recognition(modified_image_path, database, model_name=model)
        print(f"Recognized person (size: {size}, model: {model}):", recognized_person)



Testing model: VGG-Face
Verification not successful. Distance: 0.82 (Threshold: 0.68)
Verification not successful. Distance: 0.92 (Threshold: 0.68)
Verification not successful. Distance: 0.82 (Threshold: 0.68)
Verification not successful. Distance: 0.94 (Threshold: 0.68)
Verification not successful. Distance: 0.90 (Threshold: 0.68)
Recognized person (brightness: 0.1, model: VGG-Face): Unknown
Verification not successful. Distance: 0.86 (Threshold: 0.68)
Verification not successful. Distance: 0.96 (Threshold: 0.68)
Verification not successful. Distance: 0.88 (Threshold: 0.68)
Verification not successful. Distance: 0.95 (Threshold: 0.68)
Verification not successful. Distance: 0.94 (Threshold: 0.68)
Recognized person (brightness: 0.3, model: VGG-Face): Unknown
Verification successful. Distance: 0.03 (Threshold: 0.68)
Recognized person (brightness: 0.5, model: VGG-Face): trump
Verification successful. Distance: 0.08 (Threshold: 0.68)
Recognized person (brightness: 0.8, model: VGG-Face): tr